In [42]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

In [2]:
standing_url = "http://fbref.com/en/comps/9/Premier-League-Stats"

In [3]:
data = requests.get(standing_url)

In [4]:
# Parse HTML data
soup = BeautifulSoup(data.text)

In [5]:
# Selecting Stats table
standings_table = soup.select('table.stats_table')[0]

In [6]:
#finding all anchor tags in standing table
links = standings_table.find_all('a')

In [7]:
# list comprehension to get all links to teams from the table
links = [l.get('href') for l in links]

In [8]:
links = [l for l in links if '/squads/' in l]

In [9]:
# turning links into full URL "absolute links"
team_urls = [f"https://fbref.com{l}" for l in links]

In [10]:
# extracting data from URL
team_url = team_urls[0]

In [11]:
data = requests.get(team_url)

In [12]:
# creating dataframe from html
matches = pd.read_html(data.text, match="Scores & Fixtures")

In [15]:
matches[0].head()

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,xG,xGA,Poss,Attendance,Captain,Formation,Referee,Match Report,Notes
0,2021-08-07,17:15,Community Shield,FA Community Shield,Sat,Neutral,L,0,1,Leicester City,NaN,NaN,57.0,NaN,Fernandinho,4-3-3,Paul Tierney,Match Report,NaN
1,2021-08-15,16:30,Premier League,Matchweek 1,Sun,Away,L,0,1,Tottenham,1.9,1.3,64.0,58262.0,Fernandinho,4-3-3,Anthony Taylor,Match Report,NaN
2,2021-08-21,15:00,Premier League,Matchweek 2,Sat,Home,W,5,0,Norwich City,2.7,0.1,67.0,51437.0,İlkay Gündoğan,4-3-3,Graham Scott,Match Report,NaN
3,2021-08-28,12:30,Premier League,Matchweek 3,Sat,Home,W,5,0,Arsenal,3.8,0.1,80.0,52276.0,İlkay Gündoğan,4-3-3,Martin Atkinson,Match Report,NaN
4,2021-09-11,15:00,Premier League,Matchweek 4,Sat,Away,W,1,0,Leicester City,2.9,0.8,61.0,32087.0,İlkay Gündoğan,4-3-3,Paul Tierney,Match Report,NaN


### getting match shooting stats with request and pandas

In [16]:
soup = BeautifulSoup(data.text)

In [17]:
links = soup.find_all('a')

In [18]:
links = [l.get('href') for l in links]

In [19]:
# getting link with tag 'all_comps/shooting'
links = [l for l in links if l and 'all_comps/shooting' in l]

In [20]:
#link was reapeated 4 times but is the same link
links

['/en/squads/b8fd03ef/2021-2022/matchlogs/all_comps/shooting/Manchester-City-Match-Logs-All-Competitions',
 '/en/squads/b8fd03ef/2021-2022/matchlogs/all_comps/shooting/Manchester-City-Match-Logs-All-Competitions',
 '/en/squads/b8fd03ef/2021-2022/matchlogs/all_comps/shooting/Manchester-City-Match-Logs-All-Competitions',
 '/en/squads/b8fd03ef/2021-2022/matchlogs/all_comps/shooting/Manchester-City-Match-Logs-All-Competitions']

In [21]:
data = requests.get(f"https://fbref.com{links[0]}")

In [22]:
shooting = pd.read_html(data.text, match="Shooting")[0]


In [23]:
shooting.head()

For Manchester City                                                     \
                 Date   Time              Comp                Round  Day   
0          2021-08-07  17:15  Community Shield  FA Community Shield  Sat   
1          2021-08-15  16:30    Premier League          Matchweek 1  Sun   
2          2021-08-21  15:00    Premier League          Matchweek 2  Sat   
3          2021-08-28  12:30    Premier League          Matchweek 3  Sat   
4          2021-09-11  15:00    Premier League          Matchweek 4  Sat   

                                         ... Standard               Expected  \
     Venue Result GF GA        Opponent  ...     Dist   FK PK PKatt       xG   
0  Neutral      L  0  1  Leicester City  ...      NaN  NaN  0     0      NaN   
1     Away      L  0  1       Tottenham  ...     16.9  1.0  0     0      1.9   
2     Home      W  5  0    Norwich City  ...     17.3  1.0  0     0      2.7   
3     Home      W  5  0         Arsenal  ...     14.3  0.0  0     0      3.8   
4     Away      W  1  0  Leicester City  ...     14.0  0.0  0     0      2.9   

                            Unnamed: 25_level_0  
  npxG npxG/Sh G-xG np:G-xG        Match Report  
0  NaN     NaN  NaN     NaN        Match Report  
1  1.9    0.11 -1.9    -1.9        Match Report  
2  2.7    0.17  1.3     1.3        Match Report  
3  3.8    0.15  1.2     1.2        Match Report  
4  2.9    0.12 -1.9    -1.9        Match Report  

[5 rows x 26 columns]

### cleaning and merging scrapped data with pandas

In [24]:
# cleaning mutlple level indexes in shooting table
shooting.columns = shooting.columns.droplevel()

In [25]:
shooting.head()

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,...,Dist,FK,PK,PKatt,xG,npxG,npxG/Sh,G-xG,np:G-xG,Match Report
0,2021-08-07,17:15,Community Shield,FA Community Shield,Sat,Neutral,L,0,1,Leicester City,...,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,Match Report
1,2021-08-15,16:30,Premier League,Matchweek 1,Sun,Away,L,0,1,Tottenham,...,16.9,1.0,0,0,1.9,1.9,0.11,-1.9,-1.9,Match Report
2,2021-08-21,15:00,Premier League,Matchweek 2,Sat,Home,W,5,0,Norwich City,...,17.3,1.0,0,0,2.7,2.7,0.17,1.3,1.3,Match Report
3,2021-08-28,12:30,Premier League,Matchweek 3,Sat,Home,W,5,0,Arsenal,...,14.3,0.0,0,0,3.8,3.8,0.15,1.2,1.2,Match Report
4,2021-09-11,15:00,Premier League,Matchweek 4,Sat,Away,W,1,0,Leicester City,...,14.0,0.0,0,0,2.9,2.9,0.12,-1.9,-1.9,Match Report


In [27]:
# merging match and shooting dataframe

team_data = matches[0].merge(shooting[["Date", "Sh", "Dist", "FK", "PK", "PKatt"]], on="Date")

In [28]:
team_data.head()

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,...,Captain,Formation,Referee,Match Report,Notes,Sh,Dist,FK,PK,PKatt
0,2021-08-07,17:15,Community Shield,FA Community Shield,Sat,Neutral,L,0,1,Leicester City,...,Fernandinho,4-3-3,Paul Tierney,Match Report,NaN,12,NaN,NaN,0,0
1,2021-08-15,16:30,Premier League,Matchweek 1,Sun,Away,L,0,1,Tottenham,...,Fernandinho,4-3-3,Anthony Taylor,Match Report,NaN,18,16.9,1.0,0,0
2,2021-08-21,15:00,Premier League,Matchweek 2,Sat,Home,W,5,0,Norwich City,...,İlkay Gündoğan,4-3-3,Graham Scott,Match Report,NaN,16,17.3,1.0,0,0
3,2021-08-28,12:30,Premier League,Matchweek 3,Sat,Home,W,5,0,Arsenal,...,İlkay Gündoğan,4-3-3,Martin Atkinson,Match Report,NaN,25,14.3,0.0,0,0
4,2021-09-11,15:00,Premier League,Matchweek 4,Sat,Away,W,1,0,Leicester City,...,İlkay Gündoğan,4-3-3,Paul Tierney,Match Report,NaN,25,14.0,0.0,0,0


In [30]:
matches[0].shape, shooting.shape

((58, 19), (58, 26))

### scraping data for multiple season and teams with a loop

In [36]:
years = list(range(2022,2017, -1))

In [37]:
years

[2022, 2021, 2020, 2019, 2018]

In [38]:
all_matches = []

In [39]:
standing_url = "http://fbref.com/en/comps/9/Premier-League-Stats"

In [46]:
for year in years:
    data = requests.get(standing_url)
    soup = BeautifulSoup(data.text)
    standings_table = soup.select('table.stats_table')[0]

    links = [l.get('href') for l in standings_table.find_all('a')]
    links = [l for l in links if '/squads/' in l]
    team_urls = [f"https://fbref.com{l}" for l in links]

    previous_season = soup.select("a.prev")[0].get('href')
    standings_url = f"https://fbref.com{previous_season}"

    for team_url in team_urls:
        team_name = team_url.split('/')[-1].replace("Stats", "").replace("-", "")

        data = requests.get(team_url)
        matches = pd.read_html(data.text, match="Scores & Fixtures")[0]

        soup = BeautifulSoup(data.text)
        links = [l.get('href') for l in soup.find_all('a')]
        links = [l for l in links if l and 'all_comps/shooting/' in l]
        data = requests.get(f"http://fbref.com{links[0]}")
        shooting = pd.read_html(data.text, match="Shooting")[0]
        shooting.columns = shooting.columns.droplevel()

        try:
            team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")
        except ValueError:
            continue

        team_data = team_data[team_data["Comp"] == "Premier League"]
        team_data["Season"] = year
        team_data["Team"] = team_name
        all_matches.append(team_data)
        time.sleep(2)


ImportError: html5lib not found, please install it

In [ ]:
match_df = pd.concat(all_matches)

In [ ]:
match_df.columns = [c.lower() for c in match_df.columns]

In [ ]:
match_df.head()

In [ ]:
match_df.to_csv("matches.csv")